In [5]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(tf.__version__)
print('TensorFlow version:',tf.__version__)
print('Keras version:',tf.keras.__version__)

2.4.1
TensorFlow version: 2.4.1
Keras version: 2.4.0


In [6]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [7]:
classes = pd.read_csv("data/Train.csv")

In [8]:
classes.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,27,26,5,5,22,20,20,Train/20/00020_00000_00000.png
1,28,27,5,6,23,22,20,Train/20/00020_00000_00001.png
2,29,26,6,5,24,21,20,Train/20/00020_00000_00002.png
3,28,27,5,6,23,22,20,Train/20/00020_00000_00003.png
4,28,26,5,5,23,21,20,Train/20/00020_00000_00004.png


In [9]:
min_width, max_width = max(classes.Width), min(classes.Width)
min_height, max_height = max(classes.Height), min(classes.Height)

print(np.mean([min_width, max_width]))
print(np.mean([min_height, max_height]))

134.0
125.0


In [10]:
classes_no = len(classes.ClassId.unique())
print("There are {} unique classes in the dataset.".format(classes_no))

There are 43 unique classes in the dataset.


In [11]:
TARGET_SIZE = (125, 125)

#### Load a pre-trained model

In [16]:
def my_model():
    inputs = tf.keras.Input(shape=(125, 125, 3))
    x = layers.Conv2D(32, 3)(inputs)
    x = layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3)(x)
    x = layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3)(x)
    x = layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(classes_no)(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# create the base pre-trained model
base_model = my_model()

In [17]:
base_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 125, 125, 3)]     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 123, 123, 32)      896       
_________________________________________________________________
batch_normalization_5 (Batch (None, 123, 123, 32)      128       
_________________________________________________________________
tf.nn.relu_3 (TFOpLambda)    (None, 123, 123, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 61, 61, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 59, 59, 64)        18496     
_________________________________________________________________
batch_normalization_6 (Batch (None, 59, 59, 64)        256 

In [21]:
# compile the model (should be done *after* setting layers to non-trainable)
base_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy', 'categorical_crossentropy'])

Load the data and use data augmentation

In [22]:
cwd = os.getcwd()
base_dir = os.path.join(cwd, 'data')
train_path= os.path.join(base_dir, 'Train')

In [23]:
BATCH_SIZE = 20

In [24]:
# Create a data generator for the training images
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)  # val 20%

# Create a data generator for the validation images
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

#Split data to training and validation datasets
train_data = train_datagen.flow_from_directory(train_path, 
                                               target_size=TARGET_SIZE, 
                                               color_mode='rgb',
                                               batch_size=BATCH_SIZE, 
                                               class_mode='categorical',
                                               shuffle=True,
                                               subset = 'training') 

val_data = val_datagen.flow_from_directory(train_path, 
                                           target_size=TARGET_SIZE, 
                                           color_mode='rgb',
                                           batch_size=BATCH_SIZE, 
                                           class_mode='categorical',
                                           shuffle=False,
                                           subset = 'validation')

Found 31368 images belonging to 43 classes.
Found 7841 images belonging to 43 classes.


In [25]:
callback = tf.keras.callbacks.EarlyStopping(monitor='categorical_crossentropy', patience=10)

In [27]:
history = base_model.fit(
      train_data,
      steps_per_epoch= train_data.samples // BATCH_SIZE,  # One pass through entire training dataset
      epochs=100,
      validation_data=val_data,
      validation_steps= val_data.samples // BATCH_SIZE,  # One pass through entire validation dataset
      validation_freq=10,
      verbose=1)

Epoch 1/100
1568/1568 [==============================] - 138s 85ms/step - loss: 8.3553 - accuracy: 0.0362 - categorical_crossentropy: 8.3553
Epoch 2/100
1568/1568 [==============================] - 205s 130ms/step - loss: 8.4032 - accuracy: 0.0315 - categorical_crossentropy: 8.4032
Epoch 3/100
1568/1568 [==============================] - 145s 92ms/step - loss: 8.2517 - accuracy: 0.0305 - categorical_crossentropy: 8.2517
Epoch 4/100
1568/1568 [==============================] - 133s 85ms/step - loss: 8.4681 - accuracy: 0.0389 - categorical_crossentropy: 8.4681
Epoch 5/100
1568/1568 [==============================] - 139s 89ms/step - loss: 8.0778 - accuracy: 0.0557 - categorical_crossentropy: 8.0778
Epoch 6/100
1568/1568 [==============================] - 139s 88ms/step - loss: 8.2408 - accuracy: 0.0535 - categorical_crossentropy: 8.2408
Epoch 7/100
1568/1568 [==============================] - 142s 90ms/step - loss: 8.3542 - accuracy: 0.0546 - categorical_crossentropy: 8.35423s - loss: 8.

KeyboardInterrupt: 

In [1]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

NameError: name 'base_model' is not defined

In [2]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

NameError: name 'model' is not defined

In [ ]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers

history2 = model.fit(
      train_data,
      steps_per_epoch= train_data.samples // BATCH_SIZE,  # One pass through entire training dataset
      epochs=20,
      validation_data=val_data,
      validation_steps= val_data.samples // BATCH_SIZE,  # One pass through entire validation dataset
      validation_freq=10,
      verbose=1)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.save('trained_model/my_model') 

In [ ]:
model_path = os.path.join(cwd, "saved_model/my_model")
model = tf.keras.models.load_model(model_path)

In [ ]:
path = os.path.join(cwd, "data", "Test", "00010.png")
img = tf.keras.preprocessing.image.load_img(path, target_size=(125, 125))

In [ ]:
x = tf.keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)
classes = model.predict(x, batch_size=0)
plt.imshow(img)
np.argmax(classes, axis=1)